# Universal Multi-Layer Probe Inference

Test notebook for running inference across ALL probes and ALL layers

In [1]:
import sys
from pathlib import Path

# Add probes module to path
sys.path.insert(0, str(Path.cwd().parent / "src" / "probes"))

from universal_multi_layer_inference import UniversalMultiLayerInferenceEngine

/Users/ivanculo/Desktop/Projects/ment_helth/brije/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize the Engine

This loads all 450 probes (45 actions × 10 layers)

In [2]:
probes_dir = Path.cwd().parent / "data" / "probes_binary"

engine = UniversalMultiLayerInferenceEngine(
    probes_base_dir=probes_dir,
    model_name="google/gemma-3-4b-it",
    device=None  # Auto-detect
)

Detected compute device: Apple Metal Performance Shaders (MPS)
Initializing UniversalMultiLayerInferenceEngine...
  Probes base dir: /Users/ivanculo/Desktop/Projects/ment_helth/brije/data/probes_binary
  Model: google/gemma-3-4b-it
  Device: mps
  Layer range: 21-30 (10 layers)

Loading probes from all layers...
Loaded probe from /Users/ivanculo/Desktop/Projects/ment_helth/brije/data/probes_binary/layer_21/probe_abstracting.pth
Loaded probe from /Users/ivanculo/Desktop/Projects/ment_helth/brije/data/probes_binary/layer_21/probe_accepting.pth
Loaded probe from /Users/ivanculo/Desktop/Projects/ment_helth/brije/data/probes_binary/layer_21/probe_analogical_thinking.pth
Loaded probe from /Users/ivanculo/Desktop/Projects/ment_helth/brije/data/probes_binary/layer_21/probe_analyzing.pth
Loaded probe from /Users/ivanculo/Desktop/Projects/ment_helth/brije/data/probes_binary/layer_21/probe_applying.pth
Loaded probe from /Users/ivanculo/Desktop/Projects/ment_helth/brije/data/probes_binary/layer_21

`torch_dtype` is deprecated! Use `dtype` instead!


Detected vision-language model. Loading text-only (skipping vision tower)...


Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.31s/it]



✓ Initialization complete!



## Test Text #1: Analytical Reasoning

In [3]:
text1 = "I'm thinking about how to solve this complex problem step by step"

print(f"Text: {text1}\n")
print("="*80)

Text: I'm thinking about how to solve this complex problem step by step



### Mode 1: Flat ranked list (top 20 across ALL layers)

In [6]:
preds = engine.predict_all(text1, threshold=0.00001, top_k=20)

print("Top 20 predictions across all layers:\n")
for i, pred in enumerate(preds, 1):
    marker = "✓" if pred.is_active else " "
    print(f"  {marker} {i:2d}. {pred.action_name:30s} (Layer {pred.layer:2d})  {pred.confidence:.4f}")

Top 20 predictions across all layers:

  ✓  1. analyzing                      (Layer 26)  1.0000
  ✓  2. understanding                  (Layer 27)  1.0000
  ✓  3. analyzing                      (Layer 29)  1.0000
  ✓  4. emotion_understanding          (Layer 29)  1.0000
  ✓  5. understanding                  (Layer 30)  1.0000
  ✓  6. divergent_thinking             (Layer 24)  0.9961
  ✓  7. analyzing                      (Layer 22)  0.0000
     8. convergent_thinking            (Layer 30)  0.0000
     9. understanding                  (Layer 26)  0.0000
    10. convergent_thinking            (Layer 25)  0.0000
    11. convergent_thinking            (Layer 29)  0.0000
    12. creating                       (Layer 22)  0.0000
    13. self_questioning               (Layer 28)  0.0000
    14. metacognitive_regulation       (Layer 22)  0.0000
    15. analyzing                      (Layer 21)  0.0000
    16. analyzing                      (Layer 28)  0.0000
    17. applying                 

### Mode 2: By action (max confidence across layers)

In [8]:
action_preds = engine.predict_by_action(text1, threshold=0.1, aggregation="max")

print("Top 10 actions (max confidence across layers):\n")
for i, (action_name, data) in enumerate(list(action_preds.items())[:10], 1):
    if data['is_active']:
        best_layer = data['best_layer']
        aggregate = data['aggregate']
        
        # Show top 3 layers for this action
        layer_confs = sorted(data['confidences'].items(), key=lambda x: x[1], reverse=True)[:3]
        layer_str = ", ".join([f"L{l}:{c:.3f}" for l, c in layer_confs])
        
        print(f"  {i:2d}. {action_name:30s} max={aggregate:.4f} @ Layer {best_layer}")
        print(f"      Active layers: {layer_str}")

Top 10 actions (max confidence across layers):

   1. analyzing                      max=1.0000 @ Layer 26
      Active layers: L26:1.000, L29:1.000, L22:0.000
   2. understanding                  max=1.0000 @ Layer 27
      Active layers: L27:1.000, L30:1.000, L26:0.000
   3. emotion_understanding          max=1.0000 @ Layer 29
      Active layers: L29:1.000, L22:0.000, L24:0.000
   4. divergent_thinking             max=0.9961 @ Layer 24
      Active layers: L24:0.996, L22:0.000, L23:0.000


### Mode 3: By layer (which actions fire at each depth)

In [ ]:
layer_preds = engine.predict_by_layer(text1, threshold=0.1)

print("Predictions grouped by layer (top 3 actions per layer):\n")
for layer, preds in layer_preds.items():
    if preds:
        print(f"\n  Layer {layer} ({len(preds)} active actions):")
        for j, pred in enumerate(preds[:3], 1):
            print(f"    {j}. {pred.action_name:30s} {pred.confidence:.4f}")

## Test Text #2: Reflection/Metacognition

In [ ]:
text2 = "Looking back on my decision, I realize I should have considered other options"

print(f"Text: {text2}\n")
print("="*80)

preds2 = engine.predict_all(text2, threshold=0.1, top_k=15)

print("\nTop 15 predictions:\n")
for i, pred in enumerate(preds2, 1):
    marker = "✓" if pred.is_active else " "
    print(f"  {marker} {i:2d}. {pred.action_name:30s} (Layer {pred.layer:2d})  {pred.confidence:.4f}")

## Test Text #3: Empathy/Emotion

In [ ]:
text3 = "I feel deeply moved by their story and want to help them"

print(f"Text: {text3}\n")
print("="*80)

preds3 = engine.predict_all(text3, threshold=0.1, top_k=15)

print("\nTop 15 predictions:\n")
for i, pred in enumerate(preds3, 1):
    marker = "✓" if pred.is_active else " "
    print(f"  {marker} {i:2d}. {pred.action_name:30s} (Layer {pred.layer:2d})  {pred.confidence:.4f}")

## Compare Two Texts

In [ ]:
comparison = engine.compare_texts(text1, text2, top_k=10)

print("Biggest differences between Text 1 and Text 2:\n")
for i, diff in enumerate(comparison['biggest_differences'], 1):
    action = diff['action']
    t1_conf = diff['text1_confidence']
    t2_conf = diff['text2_confidence']
    delta = diff['difference']
    
    arrow = "↑" if delta > 0 else "↓"
    
    print(f"  {i:2d}. {action:30s} {arrow} {abs(delta):.4f}  (Text1: {t1_conf:.3f}, Text2: {t2_conf:.3f})")

## Interactive Testing

Try your own text!

In [ ]:
# Edit this text to test your own examples
custom_text = "Your text here..."

preds = engine.predict_all(custom_text, threshold=0.1, top_k=15)

print(f"Text: {custom_text}\n")
print("="*80)
print("\nTop 15 predictions:\n")
for i, pred in enumerate(preds, 1):
    marker = "✓" if pred.is_active else " "
    print(f"  {marker} {i:2d}. {pred.action_name:30s} (Layer {pred.layer:2d})  {pred.confidence:.4f}")

## Analyze Cross-Layer Patterns for Specific Action

In [ ]:
# Pick an action to analyze in detail
target_action = "reasoning"  # Change this to any cognitive action

text = "I'm thinking about how to solve this complex problem step by step"

action_preds = engine.predict_by_action(text, threshold=0.0, aggregation="all")

if target_action in action_preds:
    data = action_preds[target_action]
    print(f"Cross-layer analysis for '{target_action}':\n")
    print(f"Text: {text}\n")
    
    # Sort by layer
    layer_confs = sorted(data['confidences'].items())
    
    print("Confidence across layers:")
    for layer, conf in layer_confs:
        bar = "█" * int(conf * 50)
        print(f"  Layer {layer:2d}: {conf:.4f} {bar}")
    
    print(f"\nBest layer: {data['best_layer']}")
    print(f"Max confidence: {max(data['confidences'].values()):.4f}")
    print(f"Mean confidence: {sum(data['confidences'].values()) / len(data['confidences']):.4f}")
else:
    print(f"Action '{target_action}' not found")

## Batch Testing: Multiple Texts

Add as many texts as you want to test in bulk!

In [12]:
from collections import Counter, defaultdict
import numpy as np

# Add your test strings here
test_strings = [
    "The quarterly numbers look... interesting. Revenue up 12%, but margins down 3%. Customer acquisition costs rising while retention rates plateau. Something doesn't add up here.",
    "What if we completely flipped the script? Instead of chasing the same customers everyone else wants, what about targeting the segment nobody's paying attention to?",
    "Last quarter's campaign... we spent $50K on social media ads, got 200 signups, but only 15 converted. That's a 7.5% conversion rate. Industry average is 12%. We're bleeding money.",
    "That client meeting keeps replaying in my head. Sarah said 'the integration feels clunky' and I brushed it off. Now three clients have mentioned the same thing. I should have listened.",
    "My brain is scattered. Need to organize this mess: finish the Q4 budget review, prep for tomorrow's board meeting, and draft the hiring plan for next quarter. Otherwise I'll forget something crucial.",
    "Where are we on the Johnson account? Last I heard, legal was reviewing the contract. Marketing said they'd have the campaign ready by Friday. Finance needs the numbers by end of week. Everything's converging.",
    "Client feedback from Project Alpha, user research from Beta, and market analysis from Gamma. All pointing in different directions. There's a pattern here I'm not seeing yet.",
    "Option A: expand to Europe, higher risk but potentially 40% revenue growth. Option B: focus on domestic market, safer but maybe 15% growth. Both have merit. Both have downsides.",
    "The website keeps crashing during peak hours. Server logs show increased traffic, but that shouldn't cause failures. There's something else going on.",
    "The interns are lost. I'm throwing terms like 'conversion funnel' and 'attribution modeling' at them. They need the basics first - what we're trying to achieve and why.",
    "This product launch strategy feels incomplete. Maybe I should bounce ideas off the team. Fresh perspectives could reveal blind spots I'm missing.",
    "I've been assuming our target demographic is 25-35 year olds. But what if that's wrong? What if I'm basing decisions on outdated assumptions?",
    "This dashboard is overwhelming. Revenue charts, user engagement metrics, conversion rates, churn analysis. Too much noise. Need to focus on what actually matters.",
    "The current approach isn't working. Users aren't engaging with the new feature. Maybe we need to pivot. Try a different angle entirely.",
    "These customer segments look similar on paper - both tech-savvy, both high income. But their behavior patterns are completely different. What am I missing?",
    "If we launch in Q2 instead of Q1, we'd have more time for testing. But competitors might beat us to market. If we rush Q1, we risk bugs. If we wait, we risk irrelevance.",
    "The manager's email was vague: 'streamline the process.' What does that mean exactly? Reduce steps? Automate tasks? Cut costs? Need to clarify before I act.",
    "I'm recommending we increase the marketing budget by 30%. But why? Because last quarter's campaign worked? Because competitors are spending more? Need solid reasoning.",
    "Sally flagged that our pricing model doesn't account for seasonal fluctuations. She's right. Our revenue projections assume steady demand year-round. That's unrealistic.",
    "The project timeline is chaotic. Phase 1 should inform Phase 2, which should inform Phase 3. But everything's happening simultaneously. Need to map out dependencies.",
    "This market research feels biased. The methodology seems sound, but the conclusions feel predetermined. Like they found what they were looking for.",
    "The correlation between social media engagement and sales is strong. But that doesn't mean social media causes sales. Could be reverse causation, or a third factor entirely.",
    "Let's test this hypothesis: if our target users really want this feature, they'll use it within the first week. If not, we'll know it's not solving a real problem.",
    "Both theories explain the data well. Theory A focuses on user behavior, Theory B on market conditions. They're not mutually exclusive, but they emphasize different factors.",
    "This industry report cites impressive statistics, but I don't recognize the research firm. Need to verify their credibility before I base any decisions on their findings.",
    "Today's priorities are overwhelming. The client presentation, the budget review, the team meeting, the product demo. Can't do everything. Need to pick what's truly urgent.",
    "The alternative approach might be better. Current method is familiar, but the new one could be more efficient. Should we compare them side by side before deciding?",
    "Let me explain this simply: we're not making money because we're spending more to acquire customers than we earn from them. Like buying a $10 item for $15.",
    "I remember being overwhelmed by all these metrics and KPIs when I started. Jamie looks lost in the same way. Maybe I can help them understand what actually matters.",
    "Sitting here watching people interact with our app. Some scroll quickly, others pause and tap. Some get frustrated and leave. Others seem to find what they need. Patterns emerging."
]

threshold = 0.1
top_k = 20

print(f"Processing {len(test_strings)} texts...\n")
print("="*80)

for i, text in enumerate(test_strings, 1):
    print(f"\n[{i}/{len(test_strings)}] Text:")
    print(f'"{text}"')
    print("-"*80)
    
    preds = engine.predict_all(text, threshold=threshold, top_k=top_k)
    
    if preds:
        print("Top predictions:")
        for j, pred in enumerate(preds, 1):
            marker = "✓" if pred.is_active else " "
            print(f"  {marker} {j}. {pred.action_name:30s} (Layer {pred.layer:2d})  {pred.confidence:.4f}")
    else:
        print("  No predictions above threshold")

print(f"\n{'='*80}")
print(f"Batch testing complete: {len(test_strings)} texts processed")

Processing 30 texts...


[1/30] Text:
"The quarterly numbers look... interesting. Revenue up 12%, but margins down 3%. Customer acquisition costs rising while retention rates plateau. Something doesn't add up here."
--------------------------------------------------------------------------------
Top predictions:
  ✓ 1. analyzing                      (Layer 22)  1.0000
  ✓ 2. hypothesis_generation          (Layer 22)  1.0000
  ✓ 3. noticing                       (Layer 22)  1.0000
  ✓ 4. hypothesis_generation          (Layer 23)  1.0000
  ✓ 5. noticing                       (Layer 23)  1.0000
  ✓ 6. analyzing                      (Layer 24)  1.0000
  ✓ 7. noticing                       (Layer 24)  1.0000
  ✓ 8. analyzing                      (Layer 25)  1.0000
  ✓ 9. noticing                       (Layer 25)  1.0000
  ✓ 10. analyzing                      (Layer 26)  1.0000
  ✓ 11. noticing                       (Layer 26)  1.0000
  ✓ 12. analyzing                      (Layer 27)  1.0000

## Batch Testing with Summary Statistics

Process multiple texts and get aggregate statistics

In [ ]:
# Add as many test strings as you want here!
test_strings = [
    "I'm thinking about how to solve this problem",
    "Looking back, I should have done things differently",
    "I feel empathy for their situation",
    "This reminds me of a similar situation from before",
    "Let me break this down into smaller parts",
    "I need to understand the underlying principles",
    "What if we tried a completely different approach?",
    "I'm comparing these two options carefully",
    # Add more strings here...
]

# Settings
threshold = 0.1
top_k_per_text = 5  # Show top 5 predictions per text

print(f"Testing {len(test_strings)} texts\n")
print("="*80)

# Process each text
for i, text in enumerate(test_strings, 1):
    print(f"\n[{i}/{len(test_strings)}] Text: \"{text}\"")
    print("-"*80)
    
    # Get predictions
    preds = engine.predict_all(text, threshold=threshold, top_k=top_k_per_text)
    
    if preds:
        print(f"Top {len(preds)} predictions:")
        for j, pred in enumerate(preds, 1):
            marker = "✓" if pred.is_active else " "
            print(f"  {marker} {j}. {pred.action_name:25s} (L{pred.layer:2d}) {pred.confidence:.4f}")
    else:
        print("  No predictions above threshold")
    
print(f"\n{'='*80}")
print(f"Batch testing complete: {len(test_strings)} texts processed")